In [1]:
import numpy as np
import tinygrad 
from extra.datasets import fetch_cifar
import time
import cv2
start =time.monotonic()
X_train, Y_train = fetch_cifar(train=True)

In [6]:
from tinygrad.nn import Linear
from tinygrad.nn import optim
from tinygrad.tensor import Tensor

class TinyCIFAR:
    def __init__ (self):
        self.l1 = Linear(32,1024, bias=True)
        self.l2 = Linear(1024,1024,bias=True)
        self.l3 = Linear(1024,10, bias=True)

    def __call__ (self, x):
        x = self.l1(x)
        x = x.leakyrelu()
        x = self.l2(x)
        x = x.leakyrelu()
        x = self.l3(x)
        return x.log_softmax()

In [11]:
from extra.training import sparse_categorical_crossentropy
opt = optim.Adam([net.l1.weight, net.l2.weight, net.l3.weight))

net = TinyCIFAR()

Tensor.training= True

BS= 32

#run the modelieren

for step in range(100):
    samp=np.random.randint(0, X_train.shape[0], size=BS)
    batch = Tensor(X_train[samp], requires_grad=True)
    labels = Y_train[samp]
    out = net(batch)
    loss = sparse_categorical_crossentropy(out,labels)
    opt.zero_grad()
    loss.backward()
    opt.step()


50000


AssertionError: 